In [1]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py
/kaggle/input/testing-data-set/testing_data_set.csv


In [2]:
# lib import 
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# load dataset 
data = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

# idk apparently you have to choose a column as the one that is the closting price so i chose near_price i could be wrong
closing_price_column_name = 'near_price'

# yeah define the column i chose
target = data[closing_price_column_name]
data = data.drop(closing_price_column_name, axis=1)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# input features standardized 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# define lstm model 
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# reshape input data for model assuming a time series format
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# train
model.fit(X_train, y_train, epochs=10, batch_size=32)

# evaluate
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# predict
predictions = model.predict(X_test)


Epoch 1/10
130950/130950 [==============================] - 1169s 9ms/step - loss: nan
Epoch 2/10
130950/130950 [==============================] - 1128s 9ms/step - loss: nan
Epoch 3/10
130950/130950 [==============================] - 1232s 9ms/step - loss: nan
Epoch 4/10
130950/130950 [==============================] - 1305s 10ms/step - loss: nan
Epoch 5/10
130950/130950 [==============================] - 1312s 10ms/step - loss: nan
Epoch 6/10
130950/130950 [==============================] - 1238s 9ms/step - loss: nan
Epoch 7/10
130950/130950 [==============================] - 1326s 10ms/step - loss: nan
Epoch 8/10
130950/130950 [==============================] - 1373s 10ms/step - loss: nan
Epoch 9/10
130950/130950 [==============================] - 1313s 10ms/step - loss: nan
Epoch 10/10
32738/32738 [==============================] - 129s 4ms/step - loss: nan
Test Loss: nan
32738/32738 [==============================] - 126s 4ms/step
